In [1]:
import torch
from tqdm.auto import tqdm
import itertools
import random
import logging
import pickle
from os.path import expanduser
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torchvision import transforms, datasets
# Imports for plotting our resu
home = expanduser("~/Model_compression")

In [2]:
import torch.quantization
from torch.quantization import QuantStub, DeQuantStub

In [9]:
class CIFAR3(Dataset):

    def __init__(self,split="train",transform=None):
      if split=="train":
        with open("cifar10_hst_train", 'rb') as fo:
          self.data = pickle.load(fo) 
      elif split=="val":
        with open("cifar10_hst_val", 'rb') as fo:
          self.data = pickle.load(fo)
      else:
        with open("cifar10_hst_test", 'rb') as fo:
          self.data = pickle.load(fo)
      
      self.transform = transform

    def __len__(self):
        return len(self.data['labels'])

    def __getitem__(self, idx):
        
        x = self.data['images'][idx,:]
        r = x[:1024].reshape(32,32)
        g = x[1024:2048].reshape(32,32)
        b = x[2048:].reshape(32,32)
        
        x = Tensor(np.stack([r,g,b]))

        if self.transform is not None:
          x = self.transform(x)
        
        y = self.data['labels'][idx,0]
        return x,y 

In [4]:
train_transform = transforms.Compose([
        transforms.ColorJitter(),
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize(mean=[127.5, 127.5, 127.5],
                             std=[127.5, 127.5, 127.5])
    ])

test_transform = transforms.Compose([
        transforms.Normalize(mean=[127.5, 127.5, 127.5],
                             std=[127.5, 127.5, 127.5])
    ])

train_data = CIFAR3("train", transform=train_transform)
val_data = CIFAR3("val", transform=test_transform)
test_data = CIFAR3("test", transform=test_transform)

batch_size = 256
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2)
testloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)


In [10]:

def load_model(quantized_model, model):
    """ Loads in the weights into an object meant for quantization """
    state_dict = model.state_dict()
    model = model.to('cpu')
    quantized_model.load_state_dict(state_dict)

def fuse_modules(model):
    """ Fuse together convolutions/linear layers and ReLU """
    torch.quantization.fuse_modules(model, [['conv1', 'relu1'], 
                                            ['conv2', 'relu2'],
                                            ['conv3', 'relu3'],
                                            ['conv4', 'relu4'],
                                            ['fc1', 'relu5']], inplace=True)

In [11]:
class Net(nn.Module):
    def __init__(self, q = False):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
        self.fc1 = nn.Linear(4096, 512)
        self.relu5 = nn.ReLU()
        self.fc2 = nn.Linear(512, 3)
        #self.batchnorm1 = nn.BatchNorm1d(512)
        self.q = q
        if q:
          self.quant = QuantStub()
          self.dequant = DeQuantStub()
       
    def forward(self, x):
        #TODO
        if self.q:
            x = self.quant(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.pool1(self.conv2(x)))
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.pool2(x)
        x = torch.reshape(x, (-1 , 4096))
        #x = self.batchnorm1(self.fc1(x))
        x = self.fc1(x)
        x = self.relu5(x)
        x = self.fc2(x)
        if self.q:
          x = self.dequant(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [25]:
device = 'cpu'
model = Net(q=True)
# model_dict = torch.load(home+'/outputs/quant_model.pth')
# model.load_state_dict(model_dict['model_state_dict'])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=0.001)
criterion = torch.nn.CrossEntropyLoss()

device = torch.device('cpu')

model.to(device)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (relu2): ReLU()
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4): ReLU()
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (relu5): ReLU()
  (fc2): Linear(in_features=512, out_features=3, bias=True)
  (quant): QuantStub()
  (dequant): DeQuantStub()
)

In [26]:
def my_tester(model, valloader, length):
    val_acc = 0
    val_loss = 0
    model.eval()
    device = 'cpu'
    for j,input in enumerate(valloader,0):

        x = input[0].to(device)
        y = input[1].type(torch.LongTensor).to(device)

        
        out = model(x)

        loss = criterion(out,y)
        _, predicted = torch.max(out.data, 1)
        correct = (predicted == y).sum()

        val_acc += correct.item()
        val_loss += loss.item()

    val_acc /= length
    val_loss /= j
    return val_acc*100


In [27]:
loss_log = []
acc_log = []
val_acc_log = []
val_loss_log = []
best_val_acc = 0.0
for i in range(50):

  # Run an epoch of training
  train_running_loss = 0
  train_running_acc = 0
  model.train()
  for j,input in enumerate(trainloader,0):
   
    x = input[0].to(device)
    y = input[1].type(torch.LongTensor).to(device)
    out = model(x)
    loss = criterion(out,y)

    model.zero_grad()
    loss.backward()

    optimizer.step()

    _, predicted = torch.max(out.data, 1)
    correct = (predicted == y).sum()

    train_running_loss += loss.item()
    train_running_acc += correct.item()
    loss_log.append(loss.item())
    acc_log.append(correct.item()/len(y))

  train_running_loss /= j
  train_running_acc /= len(train_data)

  # Evaluate on validation
  val_acc = 0
  val_loss = 0
  model.eval()
  for j,input in enumerate(valloader,0):

    x = input[0].to(device)
    y = input[1].type(torch.LongTensor).to(device)

    
    out = model(x)

    loss = criterion(out,y)
    _, predicted = torch.max(out.data, 1)
    correct = (predicted == y).sum()

    val_acc += correct.item()
    val_loss += loss.item()

  val_acc /= len(val_data)
  val_loss /= j
  if val_acc > best_val_acc:
    best_val_acc = val_acc
    print("saving model")
    torch.save({
                'epoch': i+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, home+'/outputs/quant.pth')
    
  val_acc_log.append(val_acc)

  val_loss_log.append(val_loss)
  logging.info("[Epoch {:3}]   Loss:  {:8.4}     Train Acc:  {:8.4}%      Val Acc:  {:8.4}%".format(i,train_running_loss, train_running_acc*100,val_acc*100))


2023-06-16 20:11:07 INFO     [Epoch   0]   Loss:     1.116     Train Acc:      42.4%      Val Acc:     49.15%


saving model


2023-06-16 20:11:21 INFO     [Epoch   1]   Loss:     1.095     Train Acc:     52.79%      Val Acc:      61.1%


saving model


2023-06-16 20:11:35 INFO     [Epoch   2]   Loss:     1.035     Train Acc:     62.67%      Val Acc:     61.45%


...

2023-06-16 20:22:09 INFO     [Epoch  42]   Loss:     0.494     Train Acc:     80.98%      Val Acc:      83.0%


saving model


2023-06-16 20:22:24 INFO     [Epoch  43]   Loss:    0.4886     Train Acc:     81.51%      Val Acc:      82.4%
2023-06-16 20:22:41 INFO     [Epoch  44]   Loss:    0.4878     Train Acc:     81.13%      Val Acc:     82.45%
2023-06-16 20:22:58 INFO     [Epoch  45]   Loss:    0.4853     Train Acc:     81.32%      Val Acc:     82.35%
2023-06-16 20:23:14 INFO     [Epoch  46]   Loss:    0.4851     Train Acc:     81.88%      Val Acc:     82.15%
2023-06-16 20:23:30 INFO     [Epoch  47]   Loss:    0.4824     Train Acc:     81.55%      Val Acc:      82.6%
2023-06-16 20:23:44 INFO     [Epoch  48]   Loss:    0.4822     Train Acc:     81.44%      Val Acc:     83.15%


saving model


2023-06-16 20:24:00 INFO     [Epoch  49]   Loss:    0.4775     Train Acc:     81.69%      Val Acc:      82.8%


In [28]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

2164771


In [21]:
qnet = Net(q=True)
load_model(qnet, model)
fuse_modules(qnet)

In [24]:
import time
tic = time.time()
valaccu = my_tester(qnet, testloader, len(test_data))
tac = time.time()
print(valaccu, tac-tic)

33.33333333333333 1.3451600074768066


In [ ]:
qnet.qconfig = torch.quantization.default_qconfig
print(qnet.qconfig)
torch.quantization.prepare(qnet, inplace=True)
print('Post Training Quantization Prepare: Inserting Observers')
print('\n Conv1: After observer insertion \n\n', qnet.conv1)

my_tester(qnet, trainloader, len(train_data))
print('Post Training Quantization: Calibration done')
torch.quantization.convert(qnet, inplace=True)
print('Post Training Quantization: Convert done')
print('\n Conv1: After fusion and quantization \n\n', qnet.conv1)

In [ ]:
valaccu = my_tester(qnet, testloader, len(test_data))
print(valaccu)

In [ ]:
qnet = Net(q=True)
load_model(qnet, model)
fuse_modules(qnet)
qnet.qconfig = torch.quantization.get_default_qconfig('fbgemm')
print(qnet.qconfig)

torch.quantization.prepare(qnet, inplace=True)
va = my_tester(qnet, trainloader, len(train_data))
torch.quantization.convert(qnet, inplace=True)
valaccu = my_tester(qnet, testloader, len(test_data))
print(valaccu)

In [ ]:
torch.save({
                'epoch': 50,
                'model_state_dict': qnet.state_dict(),
                }, home+'/outputs/quant_model.pth')
torch.save({
                'epoch': 50,
                'model_state_dict': model.state_dict(),
                }, home+'/outputs/comparison_model.pth')

#8 MB vs 2 MB

In [23]:
#Quantization aware training
qnet = Net(q=True)
fuse_modules(qnet)
qnet.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
torch.quantization.prepare_qat(qnet, inplace=True)

print('\n Conv1: After fusion and quantization \n\n', qnet.conv1)
optimizer = torch.optim.Adam(qnet.parameters(), lr=1e-5, weight_decay=0.001)
criterion = torch.nn.CrossEntropyLoss()

device = torch.device('cpu')

qnet.to(device)
     


 Conv1: After fusion and quantization 

 ConvReLU2d(
  3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
  (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, qscheme=torch.per_channel_symmetric, reduce_range=False
    (activation_post_process): MovingAveragePerChannelMinMaxObserver(min_val=tensor([]), max_val=tensor([]))
  )
  (activation_post_process): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
    (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
  )
)


Net(
  (conv1): ConvReLU2d(
    3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
    (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, qscheme=torch.per_channel_symmetric, reduce_range=False
      (activation_post_process): MovingAveragePerChannelMinMaxObserver(min_val=tensor([]), max_val=tensor([]))
    )
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
      (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
    )
  )
  (relu1): Identity()
  (conv2): ConvReLU2d(
    32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
    

In [24]:
loss_log = []
acc_log = []
val_acc_log = []
val_loss_log = []
best_val_acc = 0.0
for i in range(50):

  # Run an epoch of training
  train_running_loss = 0
  train_running_acc = 0
  qnet.train()
  for j,input in enumerate(trainloader,0):
   
    x = input[0].to(device)
    y = input[1].type(torch.LongTensor).to(device)
    out = qnet(x)
    loss = criterion(out,y)

    qnet.zero_grad()
    loss.backward()

    optimizer.step()

    _, predicted = torch.max(out.data, 1)
    correct = (predicted == y).sum()

    train_running_loss += loss.item()
    train_running_acc += correct.item()
    loss_log.append(loss.item())
    acc_log.append(correct.item()/len(y))

  train_running_loss /= j
  train_running_acc /= len(train_data)

  # Evaluate on validation
  val_acc = 0
  val_loss = 0
  qnet.eval()
  for j,input in enumerate(valloader,0):

    x = input[0].to(device)
    y = input[1].type(torch.LongTensor).to(device)

    
    out = qnet(x)

    loss = criterion(out,y)
    _, predicted = torch.max(out.data, 1)
    correct = (predicted == y).sum()

    val_acc += correct.item()
    val_loss += loss.item()

  val_acc /= len(val_data)
  val_loss /= j
  if val_acc > best_val_acc:
    best_val_acc = val_acc
    print("saving model")
    torch.save({
                'epoch': i+1,
                'model_state_dict': qnet.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, home+'/outputs/qat.pth')
    
  val_acc_log.append(val_acc)

  val_loss_log.append(val_loss)
  logging.info("[Epoch {:3}]   Loss:  {:8.4}     Train Acc:  {:8.4}%      Val Acc:  {:8.4}%".format(i,train_running_loss, train_running_acc*100,val_acc*100))


2023-06-11 18:49:50 INFO     [Epoch   0]   Loss:     1.116     Train Acc:      43.8%      Val Acc:     60.75%


saving model


2023-06-11 18:50:15 INFO     [Epoch   1]   Loss:     1.104     Train Acc:     62.88%      Val Acc:      66.5%


saving model


2023-06-11 18:50:40 INFO     [Epoch   2]   Loss:     1.083     Train Acc:     65.94%      Val Acc:     68.85%


...

saving model


2023-06-11 19:07:18 INFO     [Epoch  42]   Loss:    0.4978     Train Acc:     80.54%      Val Acc:      82.1%
2023-06-11 19:07:41 INFO     [Epoch  43]   Loss:    0.4928     Train Acc:     81.06%      Val Acc:      82.2%
2023-06-11 19:08:06 INFO     [Epoch  44]   Loss:    0.4907     Train Acc:     80.95%      Val Acc:      81.9%
2023-06-11 19:08:31 INFO     [Epoch  45]   Loss:    0.4932     Train Acc:     81.15%      Val Acc:      82.5%
2023-06-11 19:08:54 INFO     [Epoch  46]   Loss:    0.4878     Train Acc:     81.05%      Val Acc:     82.95%


saving model


2023-06-11 19:09:20 INFO     [Epoch  47]   Loss:     0.484     Train Acc:      81.2%      Val Acc:      82.7%
2023-06-11 19:09:45 INFO     [Epoch  48]   Loss:    0.4832     Train Acc:     81.55%      Val Acc:      82.7%
2023-06-11 19:10:07 INFO     [Epoch  49]   Loss:    0.4821     Train Acc:     81.41%      Val Acc:      83.0%


saving model


In [34]:
import time
tic = time.perf_counter()
valaccu = my_tester(qnet, testloader, len(test_data))
toc = time.perf_counter()
print(valaccu, " orig time ", toc - tic)

81.13333333333334  orig time  1.759534425997117


In [ ]:
# quantize it without calibration (weights will not be final)
#modell.train()
# model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
# #model_fp32_fused = torch.quantization.fuse_modules(model,[['conv1', 'bn1', 'relu']])
# model_fp32_prepared = torch.quantization.prepare_qat(modell)
# model_int8 = torch.quantization.convert(model_fp32_prepared)

# # load the real state dict
# model_int8.load_state_dict(torch.load(home+'/outputs/qat.pth'))
qnet2 = Net(q=True)
load_model(qnet2, model)
fuse_modules(qnet2)
qnet2.qconfig = torch.quantization.get_default_qconfig('fbgemm')
va = my_tester(qnet2, trainloader, len(train_data))
torch.quantization.prepare(qnet2, inplace=True)
torch.quantization.convert(qnet2, inplace=True)
qnet2.load_state_dict(torch.load(home+'/outputs/quant_model.pth'))